In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Offline Handwritting Recognition

Model banyak mengadopsi dari keras example
https://keras.io/examples/vision/captcha_ocr/
https://keras.io/examples/vision/handwriting_recognition

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow import keras

AUTOTUNE = tf.data.AUTOTUNE

import matplotlib.pyplot as plt
import os
import glob
import cv2
import datetime

# Data Preparation

In [ ]:
np.random.seed(63)
tf.random.set_seed(63)

In [ ]:
train=pd.read_csv('../input/handwriting-recognition/written_name_train_v2.csv')
val=pd.read_csv('../input/handwriting-recognition/written_name_validation_v2.csv')
test=pd.read_csv('../input/handwriting-recognition/written_name_test_v2.csv')

train.dropna(inplace=True)
test.dropna(inplace=True)
val.dropna(inplace=True)

In [ ]:

train['Length']=train['IDENTITY'].apply(lambda x : len(str(x)))
max_name = 16
min_name = 2
train21=train[train['Length']>max_name]
train=train[train['Length']<=max_name]
train=train[train['Length']>=min_name]
train['IDENTITY']=train['IDENTITY'].str.upper()

val['IDENTITY']=val['IDENTITY'].str.upper()
test['IDENTITY']=test['IDENTITY'].str.upper()

In [ ]:
train21


In [ ]:
train[train['IDENTITY'] == 'UNREADABLE']

In [ ]:
train = train.loc[(train['IDENTITY'] != 'UNREADABLE')]
val = val.loc[(val['IDENTITY'] != 'UNREADABLE')]
# test = train.loc[(train['IDENTITY'] != 'UNREADABLE')]

In [ ]:
# Limit number of samples

max_data_train = 100000
max_data_val = 10000
max_data_test = 10000
train = train.iloc[:max_data_train]
val = val.iloc[:max_data_val]
test = test.iloc[:max_data_test]

In [ ]:
print(f"Total training samples: {train.shape[0]}")
print(f"Total validation samples: {val.shape[0]}")
print(f"Total test samples: {test.shape[0]}")

In [ ]:
# retrieve dataframe dataset
base_path = '../input/handwriting-recognition'

def get_image_paths_and_labels(df, status):
    paths = []
    labels = []
    data_path = os.path.join(base_path, status+'_v2', status)
    fn=df['FILENAME'].to_numpy()
    label=df['IDENTITY'].to_numpy()
    
#     df['img_path'] = df['FILENAME'].apply(lambda x : base_path+'/train_v2/train/'+x)
#     paths = df['img_path'].to_numpy()
    
    for i in range (len(fn)):
        img_path = os.path.join(data_path, fn[i])
        paths.append(img_path)
        labels.append(label[i])          
    return paths, labels
    

In [ ]:
train_img_paths, train_labels = get_image_paths_and_labels(train, 'train')
val_img_paths, val_labels = get_image_paths_and_labels(val, 'validation')
test_img_paths, test_labels = get_image_paths_and_labels(test, 'test')

In [ ]:
# Get character
characters = set()

for label in train_labels:
    for char in label:
        characters.add(char)

characters = sorted(list(characters))
print("Vocab size: ", len(characters))
print(characters)

In [ ]:
# Building character vocabulary

# Mapping characters to integers.
char_to_num = StringLookup(vocabulary=list(characters), mask_token=None)

# Mapping integers back to original characters.
num_to_char = StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

In [ ]:
t= char_to_num(['A', 'D', 'I', ' '])

In [ ]:
num_to_char(t)

In [ ]:
def image_resize(image, img_size):
    w, h = img_size
    # resize gambar tanpa mengubah rasio gambar
    # akan ditambahkan zero pading 
    image = tf.image.resize_with_pad(image, h, w)
    # Rotasi gambar 90 derajat searah jarum jam
    image = tf.transpose(image, perm=[1, 0, 2])
    image = tf.image.flip_left_right(image)
    return image


In [ ]:
batch_size = 16
padding_token = 99
image_width = 256
image_height = 64
max_len = 32


def preprocess_image(image_path, img_size=(image_width, image_height)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, 1)
    image = image_resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image


def vectorize_label(label):
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    length = tf.shape(label)[0]
    pad_amount = max_len - length
    label = tf.pad(label, paddings=[[0, pad_amount]], constant_values=padding_token)
    return label


def process_images_labels(image_path, label):
    image = preprocess_image(image_path)
    label = vectorize_label(label)
    return {"image": image, "label": label}


def prepare_dataset(image_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels)).map(
        process_images_labels, num_parallel_calls=AUTOTUNE
    )
    return dataset.batch(batch_size).cache().prefetch(AUTOTUNE)


In [ ]:
# %timeit
# %time
train_ds = prepare_dataset(train_img_paths, train_labels)
val_ds = prepare_dataset(val_img_paths, val_labels)
test_ds = prepare_dataset(test_img_paths, test_labels)

In [ ]:
# Visualisasi sampel data

for data in train_ds.take(1):
    images, labels = data["image"], data["label"]

    _, ax = plt.subplots(4, 4, figsize=(15, 8))

    for i in range(16):
        img = images[i]
        img = tf.image.flip_left_right(img)
        img = tf.transpose(img, perm=[1, 0, 2])
        img = (img * 255.0).numpy().clip(0, 255).astype(np.uint8)
        img = img[:, :, 0]

        # Gather indices where label!= padding_token.
        label = labels[i]
        indices = tf.gather(label, tf.where(tf.math.not_equal(label, padding_token)))
        # Convert to string.
        label = tf.strings.reduce_join(num_to_char(indices))
        label = label.numpy().decode("utf-8")

        ax[i // 4, i % 4].imshow(img, cmap="gray")
        ax[i // 4, i % 4].set_title(label)
        ax[i // 4, i % 4].axis("off")


plt.show()

# Build Model

In [ ]:
# Layer CTC tambahan di output 
class CTCLayer(keras.layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)
        # At test time, just return the computed predictions.
        return y_pred


def build_model():
    # Inputs to the model
    input_img = keras.Input(shape=(image_height, image_width, 1), name="image")
#     input_img = keras.Input(shape=(image_width, image_height, 1), name="image")
    labels = keras.layers.Input(name="label", shape=(None,))

    # First conv block.
    x = keras.layers.Conv2D(
        32,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv1",
    )(input_img)
    x = keras.layers.MaxPooling2D((2, 2), name="pool1")(x)

    # Second conv block.
    x = keras.layers.Conv2D(
        64,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv2",
    )(x)
    x = keras.layers.MaxPooling2D((2, 2), name="pool2")(x)

    new_shape = ((image_width // 4), (image_height // 4) * 64)
    x = keras.layers.Reshape(target_shape=new_shape, name="reshape")(x)
    x = keras.layers.Dense(64, activation="relu", name="dense1")(x)
    x = keras.layers.Dropout(0.2)(x)

    # RNNs.
    x = keras.layers.Bidirectional(
        keras.layers.LSTM(128, return_sequences=True, dropout=0.25)
    )(x)
    x = keras.layers.Bidirectional(
        keras.layers.LSTM(64, return_sequences=True, dropout=0.25)
    )(x)

    # +2 is to account for the two special tokens introduced by the CTC loss.
    # The recommendation comes here: https://git.io/J0eXP.
    # Menambahkan kelas tambahan, namun karena sudah ada kelas whitespace dan '-' tambah 1 kelas
    x = keras.layers.Dense(
        len(char_to_num.get_vocabulary()) + 1, activation="softmax", name="dense2"
    )(x)

    # Add CTC layer for calculating CTC loss at each step.
    output = CTCLayer(name="ctc_loss")(labels, x)

    # Define the model.
    model = keras.models.Model(
        inputs=[input_img, labels], outputs=output, name="handwriting_recognizion"
    )
    # Optimizer.
    opt = keras.optimizers.Adam()
    # Compile the model and return.
    model.compile(optimizer=opt)
    return model


# Get the model.
# Model untuk training
model = build_model()
model.summary()

In [ ]:
val_images = []
val_labels = []

for batch in val_ds:
    val_images.append(batch["image"])
    val_labels.append(batch["label"])

In [ ]:
# val_labels[:10]

In [ ]:

def calculate_edit_distance(labels, predictions):
    # Get a single batch and convert its labels to sparse tensors.
    sparse_labels = tf.cast(tf.sparse.from_dense(labels), dtype=tf.int64)

    # Make predictions and convert them to sparse tensors.
    input_len = np.ones(predictions.shape[0]) * predictions.shape[1]
    predictions_decoded = keras.backend.ctc_decode(
        predictions, input_length=input_len, greedy=True
    )[0][0][:, :max_len]
    sparse_predictions = tf.cast(
        tf.sparse.from_dense(predictions_decoded), dtype=tf.int64
    )

    # Compute individual edit distances and average them out.
    edit_distances = tf.edit_distance(
        sparse_predictions, sparse_labels, normalize=False
    )
    return tf.reduce_mean(edit_distances)


class EditDistanceCallback(keras.callbacks.Callback):
    def __init__(self, pred_model):
        super().__init__()
        self.prediction_model = pred_model

    def on_epoch_end(self, epoch, logs=None):
        edit_distances = []

        for i in range(len(val_images)):
            labels = val_labels[i]
            predictions = self.prediction_model.predict(val_images[i])
            edit_distances.append(calculate_edit_distance(labels, predictions).numpy())

        print(
            f"Mean edit distance for epoch {epoch + 1}: {np.mean(edit_distances):.4f}"
        )


In [ ]:
from datetime import datetime 
%time
start = datetime.now()
epochs = 5 # To get good results this should be at least 50.

model = build_model()
prediction_model = keras.models.Model(
    model.get_layer(name="image").input, model.get_layer(name="dense2").output
)
edit_distance_callback = EditDistanceCallback(prediction_model)

# Train the model.
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
#     callbacks=[edit_distance_callback],
)
%time
running_secs = (datetime.now() - start).seconds
print("Time: ", running_secs)

In [ ]:
prediction_model.save('ohr_test.h5')

# Evaluation

In [ ]:
 # A utility untuk decoding CTC greedy
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search.
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :max_len
    ]
    # Iterate over the results and get back the text.
    output_text = []
    for res in results:
        res = tf.gather(res, tf.where(tf.math.not_equal(res, -1)))
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text


#  Mengecek prediksi beberapa sampel gambar
for batch in test_ds.take(1):
    batch_images = batch["image"]
    _, ax = plt.subplots(4, 4, figsize=(15, 8))

    preds = prediction_model.predict(batch_images)
    pred_texts = decode_batch_predictions(preds)

    for i in range(16):
        img = batch_images[i]
        img = tf.image.flip_left_right(img)
        img = tf.transpose(img, perm=[1, 0, 2])
        img = (img * 255.0).numpy().clip(0, 255).astype(np.uint8)
        img = img[:, :, 0]

        title = f"Prediction: {pred_texts[i]}"
        ax[i // 4, i % 4].imshow(img, cmap="gray")
        ax[i // 4, i % 4].set_title(title)
        ax[i // 4, i % 4].axis("off")

plt.show()

In [ ]:
# pred_labels = []
# true_labels = []
# for batch in test_ds.take(10):
#     batch_images = batch["image"]
#     batch_labels = batch["label"]
# #     _, ax = plt.subplots(4, 4, figsize=(15, 8))
#     preds = prediction_model.predict(batch_images)
#     pred_label = decode_batch_predictions(preds)
#     pred_labels.append(pred_label)
    
# #     print(batch_labels.shape[0])
    
#     # Gather indices where label!= padding_token.
# #     label = labels[i]
# #     indices = tf.gather(label, tf.where(tf.math.not_equal(label, padding_token)))
# #     # Convert to string.
# #     label = tf.strings.reduce_join(num_to_char(indices))
# #     label = label.numpy().decode("utf-8")
#     true_label =[]
#     for label in batch_labels:
# #         text = ''.join([num_to_char[int(x)] for x in label])
#         indices = tf.gather(label, tf.where(tf.math.not_equal(label, padding_token)))
#         text = tf.strings.reduce_join(num_to_char(indices)).numpy().decode("utf-8")
# #         text = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
# #         output_text.append(text)
# #         orig_texts.append(text)
# #         orig_texts = decode_batch_predictions(batch_label)
#         true_label.append(text)
#     true_labels.append(true_label)
    
# #     for
# #     print(f"Prediction: {pred_texts[i]}")

In [ ]:
# for i in range(10):
#     for j in range(16):
#         print(pred_labels[i][j], true_labels[i][j])

In [ ]:
!pip install fastwer # library untuk mengecek performa model, CER dan WER

In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

In [ ]:
import fastwer #for calculating CER & WER
test["PREDICTION"] = ""
prediksi = []
start = 0
finish = batch_size-1
number_of_batches = test.shape[0]//batch_size + 1
i = 0
for batch in test_ds.take(number_of_batches):
    finish = finish
    batch_images = batch["image"]
    preds = prediction_model.predict(batch_images)
    pred_texts = decode_batch_predictions(preds)
    test.loc[start:finish,"PREDICTION"] = pred_texts
    start = start + len(pred_texts)
    finish = finish + len(pred_texts)
#     print(finish)
test_output = test[:batch_size*number_of_batches]
test_output["CER"] = test_output.apply(lambda x: fastwer.score_sent(x["PREDICTION"], x["IDENTITY"], char_level=True), axis = 1)
test_output["WER"] = test_output.apply(lambda x: fastwer.score_sent(x["PREDICTION"], x["IDENTITY"], char_level=False), axis = 1)
display(test_output)
print("CER_test_average: ", test_output["CER"].mean())
print("WER_test_average: ", test_output["WER"].mean())

10000 rows × 5 columns

CER_test_average:  9.261102059999999
WER_test_average:  33.14250001

48 x 256
CER_test_average:  7.923940249999999
WER_test_average:  28.25833335

1136
change shape
CER_test_average:  40.38193138
WER_test_average:  89.07166667

In [ ]:
# for p, (inp_value, _) in enumerate(validation_generator):
#     bs = inp_value['input_data'].shape[0]
#     X_data = inp_value['input_data']
#     labels = inp_value['input_label']
#     plt.imshow(X_data[0])
#     preds = prediction_model.predict(X_data)
#     pred_texts = decode_batch_predictions(preds)
    
    
#     orig_texts = []
#     for label in labels:
#         text = ''.join([label_to_char[int(x)] for x in label])
#         orig_texts.append(text)
        
#     for i in range(bs):
#         print(f'Ground truth: {orig_texts[i]} \t Predicted: {pred_texts[i]}')
#     break